In [ ]:
%load_ext autoreload
%autoreload 2

import astroalign as aa
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import skimage as sk

import sys
sys.path.append("/home/jprincen/astronomy/pyastro")

In [ ]:
from image_collection import ImageCollection, RawImageCollection, RgbImageCollection
from drizzle import Drizzle
from stack import align_images
from calibrate import calibrate_lights
from utils import stretch_rgb_image

In [ ]:
CAPTURE='2023-07-21'
DATA_BASE=os.path.join('/media/jprincen/HD', CAPTURE)
BIAS_DIR = os.path.join(DATA_BASE, 'biases')
DARK_DIR = os.path.join(DATA_BASE, 'darks')
FLAT_DIR = os.path.join(DATA_BASE, 'flats')
LIGHT_DIR = os.path.join(DATA_BASE, 'lights')

In [ ]:
bias_files = glob.glob(os.path.join(BIAS_DIR, "*.fits"))
dark_files = glob.glob(os.path.join(DARK_DIR, "*.fits"))
flat_files = glob.glob(os.path.join(FLAT_DIR, "*.fits"))
light_files = glob.glob(os.path.join(LIGHT_DIR, "*.fits"))

In [ ]:
biases = RawImageCollection.from_files(bias_files)
flats = RawImageCollection.from_files(flat_files)
darks = RawImageCollection.from_files(dark_files)
lights = RawImageCollection.from_files(light_files)

In [ ]:
# Use biases, darks and flats to calibrate lights
lights_c = calibrate_lights(biases, darks, flats, lights)

In [ ]:
img = lights_c[59]
p2, p98 = np.percentile(img, (2, 98))
l_rescale = sk.exposure.rescale_intensity(img, in_range=(p2, p98))
plt.imshow(l_rescale / np.max(l_rescale), cmap="gray")

In [ ]:
# debayer to RGB
rgb_lights = RgbImageCollection.from_raw_collection(lights_c/np.max(lights_c[0]), (1500, 2500), (2000, 4000))

In [ ]:
# Align images to reference image 10
aligned_lights = align_images(rgb_lights, 10)

In [ ]:
# Try different summing strategies. Median is better is there are outliers
sum_lights = aligned_lights.mean()
med_lights = aligned_lights.median()

In [ ]:
avg_rgb_scaled = stretch_rgb_image(sum_lights, 2, 98)
plt.imshow(avg_rgb_scaled)

In [ ]:
med_rgb_scaled = stretch_rgb_image(med_lights, 2, 98)
plt.imshow(med_rgb_scaled)

In [ ]:
light_i16 = (med_lights*(2**16-1)/np.max(med_lights)).astype(np.uint16)
bgr = cv2.cvtColor(light_i16, cv2.COLOR_RGB2BGR)
cv2.imwrite(os.path.join(DATA_BASE, "test_stack_med_cropped.png"), bgr, [cv2.IMWRITE_PNG_COMPRESSION, 0])

## Drizzle summing

In [ ]:
from tqdm.auto import tqdm

In [ ]:
drizzle = Drizzle(2000, 4000, 3)
images = rgb_lights
i = 0
for img in tqdm(images):
    tx, _ = aa.find_transform(img, images[10])
    if i != 50:
        drizzle.add_image(np.array(tx), img)
    i = i+1

In [ ]:
plt.imshow(stretch_rgb_image(drizzle.image.astype(np.float32)))

In [ ]:
light_i16 = (drizzle.image*(2**16-1)/np.max(drizzle.image)).astype(np.uint16)
bgr = cv2.cvtColor(light_i16, cv2.COLOR_RGB2BGR)
cv2.imwrite(os.path.join(DATA_BASE, "test_stack_drizzle.png"), bgr, [cv2.IMWRITE_PNG_COMPRESSION, 0])

# 